# Dataset Cleaning 

The main objectives of this notebook are: 
- Import dataset 
- Get familiar with the data 
- Determine if there are data quality issues 
- Resolve any data quality issues 

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
import polars as pl 
from google.cloud import bigquery
import numpy as np 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
import sys 
import os 


# Manually add path to read from another folder 
path2add = os.path.normpath(
    os.path.abspath(os.path.join(os.path.dirname("__file__"), os.path.pardir, "utils"))
)
if not (path2add in sys.path):
    sys.path.append(path2add)

from preprocessing import *

In [4]:
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [ ]:
# Create client object
client = bigquery.Client()

In [ ]:
QUERY = ("SELECT `date`, `visitStartTime`, `fullVisitorId`, `geoNetwork`.`subContinent`, `geoNetwork`.`country`,"
        "`geoNetwork`.`city`,`device`.`browser`, `device`.`operatingSystem`,`device`.`deviceCategory`,"
        "`trafficSource`.`source`, `h`.`item`.`transactionId`,`hp`.`v2ProductName`,`hp`.`v2ProductCategory` "
        "FROM `bigquery-public-data.google_analytics_sample.ga_sessions_20170731`, UNNEST(hits) AS h,"
        "UNNEST(h.product) AS hp;")

In [ ]:
run_query = client.query(QUERY)

In [ ]:
df = pl.from_arrow(run_query.to_arrow())

In [9]:
df.head()

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
i64,i64,i64,str,str,str,str,str,str,str,str,str,str
20170320,1490013196,741694360311983889,"""Southern Europe""","""Spain""","""not available in demo dataset""","""Chrome""","""Windows""","""desktop""","""google""",null,"""YouTube Hard Cover Journal""","""Home/Shop by Brand/YouTube/"""
20170320,1490075224,751716124732042493,"""Northern America""","""United States""","""New York""","""Safari""","""iOS""","""mobile""","""google""",null,"""Google Alpine Style Backpack""","""Home/Bags/"""
20170320,1490069949,3875753008535006311,"""Northern America""","""United States""","""not available in demo dataset""","""Safari (in-app)""","""iOS""","""mobile""","""m.facebook.com""",null,"""YouTube Custom Decals""","""Home/Shop by Brand/YouTube/"""
20170320,1490030549,null,"""Southern Europe""","""Spain""","""Barcelona""","""Chrome""","""Macintosh""","""desktop""","""google""",null,"""Suitcase Organizer Cubes""","""Home/Bags/"""
20170210,1486780166,1321377067218362907,"""Northern America""","""United States""","""not available in demo dataset""","""Chrome""","""Android""","""mobile""","""google""",null,"""YouTube RFID Journal""","""Home/Shop by Brand/YouTube/"""


In [10]:
df.shape

(18608748, 13)

In [11]:
df.columns

['date',
 'visitStartTime',
 'fullVisitorId',
 'subContinent',
 'country',
 'city',
 'browser',
 'operatingSystem',
 'deviceCategory',
 'source',
 'transactionId',
 'v2ProductName',
 'v2ProductCategory']

In [ ]:
df.sample()

In [ ]:
df.describe()

statistic,date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
str,str,f64,str,str,str,str,str,str,str,str,str,str,str
"""count""","""49512""",49512.0,"""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""49512""","""462""","""49512""","""49512"""
"""null_count""","""0""",0.0,"""0""","""0""","""0""","""0""","""0""","""0""","""0""","""0""","""49050""","""0""","""0"""
"""mean""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""std""",null,21502.167394,null,null,null,null,null,null,null,null,null,null,null
"""min""","""20170731""",1.5015e9,"""0002457163364254438""","""Australasia""","""Algeria""","""(not set)""","""Amazon Silk""","""(not set)""","""desktop""","""(direct)""","""ORD201707311786""","""1 oz Hand Sanitizer""","""${escCatTitle}"""
"""25%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""50%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""75%""",null,1.5015e9,null,null,null,null,null,null,null,null,null,null,null
"""max""","""20170731""",1.5016e9,"""9997362993085245352""","""Western Europe""","""Vietnam""","""not available in demo dataset""","""YaBrowser""","""iOS""","""tablet""","""youtube.com""","""ORD201707312663""","""YouTube Youth Short Sleeve Tee…","""Wearables/Men's T-Shirts/"""


In [ ]:
df.select(pl.all().is_null().sum())

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,1298455,0,0,0,0,0,0,0,18534291,0,0


In [ ]:
df.filter(pl.col('transactionId') != 'null')

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
i64,i64,i64,str,str,str,str,str,str,str,str,str,str
20170226,1488137668,5149788969578895545,"""Northern America""","""United States""","""Mountain View""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201702261432""","""Google Doodle Decal""","""${productitem.product.origCatN…"
20170216,1487267605,2517364798811740879,"""Northern America""","""United States""","""not available in demo dataset""","""Chrome""","""Windows""","""desktop""","""(direct)""","""ORD201702162343""","""Google Men's Lightweight Micro…","""Apparel"""
20170730,1501437710,4981679382824177253,"""Northern America""","""United States""","""San Francisco""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201707301706""","""Google Power Bank""","""Electronics"""
20170619,1497936634,614324104402254023,"""Eastern Asia""","""South Korea""","""not available in demo dataset""","""Safari""","""Macintosh""","""desktop""","""(direct)""","""ORD201706192417""","""Google Youth Short Sleeve T-sh…","""Apparel"""
20170605,1496692659,5696607948314871944,"""Northern America""","""United States""","""not available in demo dataset""","""Chrome""","""Macintosh""","""desktop""","""chat.google.com""","""ORD201706052682""","""Android Men's Outerstellar Sho…","""Apparel"""
…,…,…,…,…,…,…,…,…,…,…,…,…
20161220,1482296712,869668043543534560,"""Northern America""","""United States""","""New York""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201612202885""","""YouTube Twill Cap""","""Apparel"""
20161220,1482239514,1118628644000079833,"""Eastern Asia""","""Taiwan""","""not available in demo dataset""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201612203114""","""Google Men's Pullover Hoodie G…","""Apparel"""
20161220,1482264201,null,"""Northern America""","""United States""","""Chicago""","""Chrome""","""Macintosh""","""desktop""","""(direct)""","""ORD201612203145""","""Google Spiral Journal with Pen""","""Office"""


In [ ]:
df_cleaned = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
df_cleaned.head()

SchemaError: invalid series dtype: expected `String`, got `i64`

In [ ]:
df_cleaned = df_cleaned.with_columns(pl.when(pl.col('transactionId') != 'null').then(1).otherwise(0).alias("transactionId"))

In [ ]:
category_means = df_cleaned.group_by(pl.col('v2ProductName')).agg(
        pl.col('transactionId').mean()
)

: 

In [ ]:
for i in df_cleaned.columns:
    print(i)

date
visitStartTime
fullVisitorId
subContinent
country
city
browser
operatingSystem
deviceCategory
source
transactionId
v2ProductName
v2ProductCategory


In [ ]:
df_encoded = df_cleaned

In [ ]:
for i in df_encoded.columns:
    if i == 'date' or i == 'visitStartTime' or i == 'fullVisitorId' or i == 'transactionId':
        continue 
    print(i) 
    category_means = df_encoded.group_by(pl.col(f'{i}')).agg(
        pl.col('transactionId').mean()
    )
    df_encoded = df_encoded.join(category_means, on=f'{i}', how='left')
    df_encoded = df_encoded.with_columns(pl.col("transactionId_right").alias(f"{i}")).drop("transactionId_right")


subContinent
country
city
browser
operatingSystem
deviceCategory
source
v2ProductName
v2ProductCategory


In [ ]:
df_encoded

date,visitStartTime,fullVisitorId,subContinent,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductName,v2ProductCategory
date,i64,str,f64,f64,f64,f64,f64,f64,f64,i32,f64,f64
2017-07-31,1501520683,"""2219384770970157334""",0.0,0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0,0.0
2017-07-31,1501520683,"""2219384770970157334""",0.0,0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0,0.0
2017-07-31,1501520683,"""2219384770970157334""",0.0,0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.0,0.0
2017-07-31,1501520683,"""2219384770970157334""",0.0,0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.008,0.0
2017-07-31,1501520683,"""2219384770970157334""",0.0,0.0,0.006208,0.001795,0.005962,0.011631,0.003665,0,0.00738,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…
2017-07-31,1501537217,"""7483600664917507409""",0.013947,0.014312,0.013889,0.011785,0.01692,0.011631,0.020215,1,0.007491,0.254777
2017-07-31,1501537217,"""7483600664917507409""",0.013947,0.014312,0.013889,0.011785,0.01692,0.011631,0.020215,1,0.004301,0.262
2017-07-31,1501537217,"""7483600664917507409""",0.013947,0.014312,0.013889,0.011785,0.01692,0.011631,0.020215,1,0.054054,0.262


# Encoding the df

In [14]:
encoded_df = target_encoding(df)
encoded_df

: 

In [ ]:
# encoded_df = label_encoding(df)
# encoded_df

In [ ]:
user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
    pl.col('country'), pl.col('city'), 
    pl.col('browser'),pl.col('operatingSystem'),pl.col('deviceCategory'),
    pl.col('source'),pl.col('transactionId'),pl.col('v2ProductCategory')
)
user_profiles

fullVisitorId,country,city,browser,operatingSystem,deviceCategory,source,transactionId,v2ProductCategory
str,list[f64],list[f64],list[f64],list[f64],list[f64],list[f64],list[i32],list[f64]
"""9187441282012910839""","[0.014312, 0.014312, … 0.014312]","[0.006208, 0.006208, … 0.006208]","[0.011785, 0.011785, … 0.011785]","[0.006482, 0.006482, … 0.006482]","[0.003792, 0.003792, … 0.003792]","[0.020215, 0.020215, … 0.020215]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""872083358799320528""","[0.0, 0.0, … 0.0]","[0.006208, 0.006208, … 0.006208]","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.011631, 0.011631, … 0.011631]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""9442478693509373201""","[0.014312, 0.014312, 0.014312]","[0.020138, 0.020138, 0.020138]","[0.011785, 0.011785, 0.011785]","[0.01692, 0.01692, 0.01692]","[0.011631, 0.011631, 0.011631]","[0.020215, 0.020215, 0.020215]","[0, 0, 0]","[0.0, 0.0, 0.0]"
"""6658001868255655798""","[0.003591, 0.003591, … 0.003591]","[0.006208, 0.006208, … 0.006208]","[0.000797, 0.000797, … 0.000797]","[0.01692, 0.01692, … 0.01692]","[0.011631, 0.011631, … 0.011631]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""9445171435422230535""","[0.014312, 0.014312, … 0.014312]","[0.078788, 0.078788, … 0.078788]","[0.011785, 0.011785, … 0.011785]","[0.01692, 0.01692, … 0.01692]","[0.011631, 0.011631, … 0.011631]","[0.020215, 0.020215, … 0.020215]","[0, 0, … 1]","[0.0, 0.0, … 0.262]"
…,…,…,…,…,…,…,…,…
"""5825582381392813258""","[0.0, 0.0, … 0.0]","[0.006208, 0.006208, … 0.006208]","[0.000797, 0.000797, … 0.000797]","[0.000566, 0.000566, … 0.000566]","[0.003792, 0.003792, … 0.003792]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""6869672017204231012""","[0.014312, 0.014312, … 0.014312]","[0.013889, 0.013889, … 0.013889]","[0.011785, 0.011785, … 0.011785]","[0.01692, 0.01692, … 0.01692]","[0.011631, 0.011631, … 0.011631]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"
"""0037643978962092017""","[0.0, 0.0, … 0.0]","[0.0, 0.0, … 0.0]","[0.011785, 0.011785, … 0.011785]","[0.005962, 0.005962, … 0.005962]","[0.011631, 0.011631, … 0.011631]","[0.003665, 0.003665, … 0.003665]","[0, 0, … 0]","[0.0, 0.0, … 0.0]"


In [ ]:
user_profiles = user_profiles.filter(pl.col('fullVisitorId') != '327480040583975209')

In [ ]:
user_profiles_no_id = user_profiles.drop('fullVisitorId')
user_profiles_no_id = user_profiles_no_id.to_numpy()
user_profiles_no_id

array([[array([0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195, 0.01431195, 0.01431195, 0.01431195,
               0.01431195, 0.01431195])                                   ,
        array([0.00620763, 0.00620763, 0.00620763, 0.00620763, 0.00620763,
               0.00620763, 0.00620763, 0.00620763, 0.00620763, 0.00620763,
               0.00620763, 0.00620763, 0.00620763, 0.00620763, 0.00620763,
               0.00620763, 0.00620763])                                   ,
        array([0.01178473, 0.01178473, 0.01178473, 0.01178473, 0.01178473,
               0.01178473, 0.01178473, 0.01178473, 0.01178473, 0.01178473,
               0.01178473, 0.01178473, 0.01178473, 0.01178473, 0.01178473,
               0.01178473, 0.01178473])                                   ,
        ...,
        array([0.02021548, 0.02021548, 0.02021548, 0.02021548, 0.02021548,
         

In [ ]:
active_users = encoded_df.drop('fullVisitorId')
active_users = active_users.to_numpy()
active_users

array([[0.        , 0.00620763, 0.00179533, ..., 0.00366492, 0.        ,
        0.        ],
       [0.        , 0.00620763, 0.00179533, ..., 0.00366492, 0.        ,
        0.        ],
       [0.        , 0.00620763, 0.00179533, ..., 0.00366492, 0.        ,
        0.        ],
       ...,
       [0.01431195, 0.01388889, 0.01178473, ..., 0.02021548, 1.        ,
        0.262     ],
       [0.01431195, 0.01388889, 0.01178473, ..., 0.02021548, 1.        ,
        0.262     ],
       [0.01431195, 0.01388889, 0.01178473, ..., 0.02021548, 0.        ,
        0.        ]])

In [ ]:
test_users = list(encoded_df.filter(pl.col('fullVisitorId') == '327480040583975209').row(-1))[1:]
test_users

[0.0,
 0.0,
 0.0017953321364452424,
 0.0059616381544841885,
 0.011630818887559475,
 0.0,
 0,
 0.0]

In [ ]:
similarities = calculate_similarity(user_profiles, user_profiles_no_id, test_users)
similarities

{'9187441282012910839': 0.2705794506835526,
 '872083358799320528': 0.7493655976806098,
 '9442478693509373201': 0.4907527841443386,
 '6658001868255655798': 0.8160014198521283,
 '9445171435422230535': 0.18535188152515927,
 '2304621907222400357': 0.8268949532328057,
 '2529360978386997334': 0.6346243619758429,
 '400568069007679938': 0.61139335646929,
 '3002991669283191017': 0.27057945068355255,
 '1045476012250361691': 0.6187729409745498,
 '3171605612678347008': 0.22474107068524382,
 '3589030066112061994': 0.0960169568581351,
 '7204212537165888128': 0.37260638658658957,
 '4028772550968575969': 0.7651852963288334,
 '3120558056452204030': 0.6912336240746823,
 '0847532481011580098': 0.8141680477959067,
 '2935670120193429583': 0.3873086046971663,
 '2934232686348529280': 0.9635111916051309,
 '6219755866181453974': 0.7798026663496941,
 '5433213914391964673': 0.8061424496083996,
 '2155563753284265170': 0.4907527841443384,
 '4584186972806797440': 0.6333125560967928,
 '7445235885559107095': 0.214327

In [ ]:
sorted_dict = sorted(similarities.items(), key=lambda x: x[1])
sorted_dict[-5:]

[('4682317243688313274', 0.9635111916051311),
 ('4526419438777992681', 0.9635111916051312),
 ('9306329208927979354', 0.9897115733342824),
 ('8028335945205964246', 0.9897115733342824),
 ('8302271511945065814', 0.990696750000991)]

In [ ]:
actual_bought = df.filter((pl.col('fullVisitorId') == '0345672699449577691') & (pl.col('transactionId') != 'null')).select(pl.col('v2ProductCategory')).to_series().to_list()
actual_bought

['Headgear',
 'Apparel',
 'Apparel',
 'Apparel',
 'Headgear',
 'Apparel',
 'Apparel',
 'Apparel']

In [ ]:
rec = recommendation(df,sorted_dict)
rec

['Apparel', 'Apparel', 'Lifestyle', 'Apparel', 'Apparel']

In [ ]:
precision(actual_bought, rec, 3)

0.5

In [ ]:
df.write_csv("../data/cleaned_google_analytics.csv")

### Data Pre-processing Pipeline

In [ ]:
def preprocessing_pipeline(df: pl.DataFrame) -> pl.DataFrame:
    """
    Args:
        df : pl.DataFrame

    Returns:
        pl.DataFrames: pre-processed dataframe
    """

    df = df.with_columns(pl.col('date').str.strptime(pl.Date, format="%Y%m%d"))
    encoded_df = encoding(df)
    
    user_profiles = encoded_df.group_by(pl.col('fullVisitorId')).agg(
        pl.col('country'), pl.col('browser'),
        pl.col('transactionId'), pl.col('v2ProductName')
    )
    user_profiles_no_id = user_profiles.drop('fullVisitorId')
    user_profiles_no_id = user_profiles_no_id.to_numpy()
    active_users = encoded_df.drop('fullVisitorId')
    active_users = active_users.to_numpy()

    similarities = calculate_similarity(user_profiles, user_profiles_no_id, active_users)
    sorted_dict = sorted(similarities.items(), key=lambda x: x[1])

    
    df.write_csv("../data/cleaned_google_analytics.csv")
    return recommendation(df,sorted_dict) 

In [ ]:
# preprocessing_pipeline(df)

# Conclusions 
- 